Arbol elemental con libreria  rpart

#Debe tener instaladas las librerias  data.table  ,  rpart   y rpart.plot

In [1]:
#cargo las librerias que necesito
require("data.table")
require("rpart")
require("rpart.plot")

Loading required package: data.table
Warning message:
"package 'data.table' was built under R version 3.6.3"Loading required package: rpart
Warning message:
"package 'rpart' was built under R version 3.6.3"Loading required package: rpart.plot
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'rpart.plot'"

In [2]:
#Aqui se debe poner la carpeta de SU computadora local
setwd("C:\\Users\\Tomás García\\Documents\\MD")  #Establezco el Working Directory

cargo los datos de 202009 que es donde voy a ENTRENAR el modelo

In [3]:

dtrain  <- fread("./datasets/paquete_premium_202011.csv")

+ Max depth es la máxima profundidad que dmite la librería
+ min split es mejor tener 2, porque es el mínimo que puede dividir, si le pongo 3 lo limito
+ mínima cantidad de resgitros del hijo en el árbol para que yo considere al hijo

genero el modelo,  aqui se construye el arbol

In [44]:
modelo  <- rpart("clase_ternaria ~ .",
                 data = dtrain,
                 xval=0,
                 cp=        -0.3,  #esto significa no limitar la complejidad de los splits
                 minsplit = 200,   #minima cantidad de registros para que se haga el split
                 minbucket = 1,   #tamaño minimo de una hoja
                 maxdepth = 10 )  #PRUEBO  una profundidad de 10

Ahora grafico el arbol, va a ser muy frondoso porque la profundidad es de 10

In [ ]:
prp(modelo, extra=101, digits=5, branch=1, type=4, varlen=0, faclen=0)

In [ ]:
#Ahora aplico al modelo  a los datos de 202101  y genero la salida para kaggle

#cargo los datos de 202011, que es donde voy a APLICAR el modelo

In [37]:
dapply  <- fread("./datasets/paquete_premium_202101.csv")

In [38]:
prediccion  <- predict( modelo, dapply , type = "prob") #aplico el modelo

prediccion es una matriz con TRES columnas, llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
cada columna es el vector de probabilidades 

agrego a dapply una columna nueva que es la probabilidad de BAJA+2

In [39]:
dapply[ , prob_baja2 := prediccion[, "BAJA+2"] ]

solo le envio estimulo a los registros con probabilidad de BAJA+2 mayor  a  1/60

In [40]:
dapply[ , Predicted  := as.numeric(prob_baja2 > 1/60) ]

genero un dataset con las dos columnas que me interesan

In [41]:
entrega  <- dapply[   , list(numero_de_cliente, Predicted) ] #genero la salida

#Finalmente genero el archivo para Kaggle

In [ ]:
#creo la carpeta donde va el experimento

In [42]:
dir.create( "./labo/exp/" ) 
dir.create( "./labo/exp/KA2002" ) 

Warning message in dir.create("./labo/exp/"):
"'.\labo\exp' already exists"Warning message in dir.create("./labo/exp/KA2002"):
"'.\labo\exp\KA2002' already exists"

grabo el dataset

In [43]:
fwrite( entrega, 
        file= "./labo/exp/KA2002/K102_001.csv", 
        sep= "," )